<a href="https://colab.research.google.com/github/FlyFish0203/flyfish/blob/main/%E6%9C%9F%E6%9C%AB%E5%A0%B1%E5%91%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**題目**
**line機器人生成小短文**

##**引言**##
因為有時候會需要用到講故事的部分，但又很臨時，可能是不同的廠景與人物，因此希望可以利用主角特徵，來生成一篇關於她的小故事。

#**特色**
- 利用一段文字可以變成一篇短文
- 可以根據形容詞來去修正這篇短文的風格

In [18]:
!pip install -q flask line-bot-sdk openai google-generativeai pyngrok flask_ngrok

In [19]:
# ✅ 匯入套件
from flask import Flask, request, abort
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
from google.colab import userdata
import google.generativeai as genai

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage
)


In [20]:
# ✅ 設定 Gemini API
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash')

# ✅ 啟動 Flask + Ngrok
app = Flask(__name__)
run_with_ngrok(app)

# ✅ LINE Bot 設定
configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))


In [21]:
def ask_gemini_to_write_story(user_text):
    # 可支援的標籤與對應風格描述
    styles = {
        "#搞笑": "用幽默搞笑的風格",
        "#浪漫": "用浪漫抒情的風格",
        "#驚悚": "用驚悚懸疑的風格",
        "#童話": "用童話故事的語氣",
    }

    # 預設風格
    selected_style = "用文學敘事的方式"

    # 檢查有無風格標籤
    for tag, style in styles.items():
        if tag in user_text:
            selected_style = style
            user_text = user_text.replace(tag, "").strip()

    # 提交給 Gemini 的 prompt
    prompt = f"{selected_style}，根據以下主題創作一篇約150字的小故事：{user_text}"

    # 呼叫 Gemini API
    response = model.generate_content(prompt)
    return response.text


In [22]:
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_input = event.message.text
    story = ask_gemini_to_write_story(user_input)

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=story)]
            )
        )


In [24]:
ngrok_start()
if __name__=='__main__':
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bffd-34-16-247-218.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
 * Running on http://bffd-34-16-247-218.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:35:39] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:35:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2025 14:36:19] "POST /callback HTTP/1.1" 200 -
